# Passer d'un code série à un code parallèle
## Première étape: Comprendre ce qui rend les choses plus rapides et ce qui ne le fait pas
Dans certains cas, il est possible d'accélérer le code série en faisant certaines petites modifications. Par exemple, R n'est pas particulièrement efficace à gérer ce qu'il se passe en mémoire. Le concept de "pré-allocation" de la mémoire est crucial dans R. Prenons par exemple la construction d'une table par itérations. Si nous voulons remplir un tableau de valeurs associées à différents spécimens par exemple, une étape logique serait de faire une boucle qui créerait une table par spécimen, pour ajouter les lignes créées à la suite dans le tableau. Ex:



In [ ]:
specimens <- list(seq(1,50,1))
jours <- 10

In [ ]:
table.specimen <- data.frame(ID=integer(), jour=integer(), hauteur = numeric(), masse = numeric(), BMI=numeric())

for(i in specimens){ # une table par specimen
  table.temp <- data.frame(ID=integer(), jour=integer(), hauteur = numeric(), masse = numeric(), BMI=numeric())
  for(j in 1:jours){ # 10 jours de mesures
    haut <- runif(n = 1, min = 5.4, max = 6.2)
    mass <- sample(98:160, 1)
    BMI <- haut/mass
    temp2 <- data.frame(ID=i, jour=j, hauteur = haut, masse = mass, BMI=BMI)
    table.temp <- rbind(table.temp, temp2)
  }
  table.specimen <- rbind(table.specimen, table.temp)
}

In [ ]:
table.alloc1 <- as.data.frame(matrix(ncol=5, nrow=length(specimens)*jours)) # nombre de lignes = nb specimens *nb jours
names(table.alloc1) <- c("ID", "jour", "hauteur", "masse", "BMI")
ligne <- 1
for(i in specimens){ # une table par specimen
  for(j in 1:jours){ # 10 jours de mesures
    haut <- runif(n = 1, min = 5.4, max = 6.2)
    mass <- sample(98:160, 1)
    BMI <- haut/mass
    table.alloc1[ligne,] <- c(i,j,haut,mass,BMI)

    ligne <- ligne+1
  }
}

In [ ]:
nb.times <- length(specimens)*jours
table.alloc2 <- data.frame(ID=rep(specimens, jours), 
                           jour=rep(1:jours, length(specimens)), 
                           hauteur = replicate(1,runif(min= 5.4, max= 6.2,n=nb.times)), 
                           masse = replicate(1,sample(98:160, nb.times, replace = TRUE)), 
                           BMI=numeric(length = nb.times))
table.alloc2$BMI <- table.alloc2$hauteur/table.alloc2$masse

## Deuxième étape, faire le profilage
Les 3 options peuvent sembler équivalentes, mais le profilage permet de trouver les zones où elles diffèrent. En effet, un profileur permet d'inspecter un code R pour déterminer à quoi le temps et les ressources servent.

Pour commencer, nous allons utiliser un profileur nommé profvis.

In [ ]:
library(profvis)

L'interface du profileur contient le code en haut et les graphiques en bas. L'axe horizontal représente le temps en millisecondes et l'axe vertical représente chaque étape du code. Pour 10 jours, vous ne verrez potentiellement même pas de graphique pour l'option 3, puisqu'elle est très rapide.

On peut aussi voir la mémoire qui est utilisée pour chaque partie du code. Prenez du temps pour inspecter chacun des graphiques et les comparer. Ajustez le nombre de jours à la hausse.

Selon le code, il serait possible que des parties n'apparaîssent pas dans le profileur, par exemple s'il n'y a pas accès.

In [ ]:
profvis({
table.specimen <- data.frame(ID=integer(), jour=integer(), hauteur = numeric(), masse = numeric(), BMI=numeric())

for(i in specimens){ # une table par specimen
  table.temp <- data.frame(ID=integer(), jour=integer(), hauteur = numeric(), masse = numeric(), BMI=numeric())
  for(j in 1:jours){ # 10 jours de mesures
    haut <- runif(n = 1, min = 5.4, max = 6.2)
    mass <- sample(98:160, 1)
    BMI <- haut/mass
    temp2 <- data.frame(ID=i, jour=j, hauteur = haut, masse = mass, BMI=BMI)
    table.temp <- rbind(table.temp, temp2)
  }
  table.specimen <- rbind(table.specimen, table.temp)
}
})

In [ ]:
profvis({
    table.alloc1 <- as.data.frame(matrix(ncol=5, nrow=length(specimens)*jours)) # nombre de lignes = nb specimens *nb jours
names(table.alloc1) <- c("ID", "jour", "hauteur", "masse", "BMI")
ligne <- 1
for(i in specimens){ # une table par specimen
  for(j in 1:jours){ # 10 jours de mesures
    haut <- runif(n = 1, min = 5.4, max = 6.2)
    mass <- sample(98:160, 1)
    BMI <- haut/mass
    table.alloc1[ligne,] <- c(i,j,haut,mass,BMI)

    ligne <- ligne+1
  }
}
    })

In [ ]:
profvis({
    nb.times <- length(specimens)*jours
table.alloc2 <- data.frame(ID=rep(specimens, jours), 
                           jour=rep(1:jours, length(specimens)), 
                           hauteur = replicate(1,runif(min= 5.4, max= 6.2,n=nb.times)), 
                           masse = replicate(1,sample(98:160, nb.times, replace = TRUE)), 
                           BMI=numeric(length = nb.times))
table.alloc2$BMI <- table.alloc2$hauteur/table.alloc2$masse
})

# Que pourrions nous faire pour améliorer le code?
- passer de data.frame à matrix
- passer de data.frame à une liste de vecteurs
- certaines fonctions peuvent être accélérées par des apply

# Lorsqu'il est devenu impossible d'accélérer le code
Pour des tâches rapides, les moyens conventionnels de réécriture du code permettent d'aller chercher de meilleures performances. Cependant, lorsque tout a été essayé et qu'on ne peut contourner le fait que nous avons affaire à des centaines de modèles ou des millions d'individus, l'utilisation de listes ou de matrices ne sera qu'une goutte d'eau dans l'océan de notre code. C'est à ce moment que le calcul parallèle entre en jeu!